In [ ]:
from qiskit import Aer, IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, execute
import socket, math

HOST = '127.0.0.1'
PORT = 64242

def quantumProcessing(data):
    device = Aer.get_backend('qasm_simulator')
    # set the number of samples used for statistics
    shots = 2048
    print(data)
    dataList = data.split(";")
    dataList.pop()
    
    #Parameters
    nbPlayer = 2
    nbOfquanticToken = 8
    TokenPos = dataList[0:nbPlayer*nbOfquanticToken]
    
    #create grid
    qc = []
    
    for player in range(2):
        # now to set up the quantum program to simulate the grid for this player
        # set up registers and program
        q = QuantumRegister(8)
        c = ClassicalRegister(8)
        qc.append(QuantumCircuit(q, c))

        # Check The position of the unit
        #for position in range(2):
            #THERE IS TO CHECK THE SPECIAL OF OUR UNITS
                
        # Finally, measure them
        for position in range(2):
            qc[player].measure(q[position], c[position])

    # compile and run the quantum program
    job = execute(qc, backend=device, shots=shots)
    # and extract data
    data = ""
    for player in range(2):
        res = job.result().get_counts(qc[player])
        for key, value in res.items():
            data += key
            data += ";"
            data += str(value)
            data += ";"
        data += "!"
    print(data)
    return data

while(True):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            try:
                while True:
                    data = conn.recv(2048)
                    if not data:
                        break
                    data = quantumProcess(data.decode())
                    conn.sendall(data.encode())
            except:
                print("error")